In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

config		LICENSE		README.md	  test_uavid.py      train.py
data		lightning_logs	requirements.txt  test_vaihingen.py
fig_ex		model_weights	test_loveda.py	  Test.zip
Greenify.ipynb	network		test_potsdam.py   tools


In [ ]:

%cd /content/drive/MyDrive/SFA-Net-main

/content/drive/.shortcut-targets-by-id/10heZNhrYJq_xt4fs5qCt9VPt9p7yiRkq/SFA-Net-main


In [ ]:
!python train.py -c config/loveda/sfanet.py

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
2025-06-05 10:18:35.647504: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-05 10:18:35.666523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749118715.689313    6865 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749118715.695876    6865 cuda_bla

In [ ]:
!pip install pytorch_lightning
!pip install addict
!pip install -r requirements.txt
!pip install pytorch_model_summary
!pip install torchinfo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
import os
import shutil

def merge_train_val(base_dir):
    split_dirs = ['Train', 'Val']
    regions = ['Urban', 'Rural']
    subfolders = ['images_png', 'masks_png', 'masks_png_convert', 'masks_png_convert_rgb']
    output_root = os.path.join(base_dir, 'train_val')

    for region in regions:
        for subfolder in subfolders:
            out_path = os.path.join(output_root, region, subfolder)
            os.makedirs(out_path, exist_ok=True)

    for split in split_dirs:
        for region in regions:
            for subfolder in subfolders:
                src_path = os.path.join(base_dir, split, region, subfolder)
                dst_path = os.path.join(output_root, region, subfolder)

                if not os.path.exists(src_path):
                    print(f"Warning: {src_path} not found. Skipping...")
                    continue

                for fname in os.listdir(src_path):
                    src_file = os.path.join(src_path, fname)
                    dst_file = os.path.join(dst_path, fname)

                    if not os.path.exists(dst_file):  # avoid overwriting
                        shutil.copy2(src_file, dst_file)
                    else:
                        print(f"Skipped duplicate: {dst_file}")

    print("✅ Merging completed.")

# Example usage
merge_train_val("/content/drive/MyDrive/SFA-Net-main/data/LoveDA")
print("Ho gaya")

In [ ]:
import os

def clean_unmatched_files(img_dir, mask_dir):
    img_files = sorted(os.listdir(img_dir))
    mask_files = sorted(os.listdir(mask_dir))

    img_names = set(os.path.splitext(f)[0] for f in img_files)
    mask_names = set(os.path.splitext(f)[0] for f in mask_files)

    unmatched_imgs = img_names - mask_names
    unmatched_masks = mask_names - img_names

    print(f"Unmatched images: {len(unmatched_imgs)}")
    print(f"Unmatched masks: {len(unmatched_masks)}")

    # Delete unmatched images
    for name in unmatched_imgs:
        path = os.path.join(img_dir, f"{name}.png")
        if os.path.exists(path):
            os.remove(path)
            print(f"Deleted image: {path}")

    # Delete unmatched masks
    for name in unmatched_masks:
        path = os.path.join(mask_dir, f"{name}.png")
        if os.path.exists(path):
            os.remove(path)
            print(f"Deleted mask: {path}")

# Paths
urban_img_dir = "/content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/images_png"
urban_mask_dir = "/content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/masks_png_convert"

rural_img_dir = "/content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Rural/images_png"
rural_mask_dir = "/content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Rural/masks_png_convert"

# Run cleanup
clean_unmatched_files(urban_img_dir, urban_mask_dir)
clean_unmatched_files(rural_img_dir, rural_mask_dir)


Unmatched images: 14
Unmatched masks: 0
Deleted image: /content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/images_png/3614.png
Deleted image: /content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/images_png/3531.png
Deleted image: /content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/images_png/3520.png
Deleted image: /content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/images_png/3622.png
Deleted image: /content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/images_png/3536.png
Deleted image: /content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/images_png/3547.png
Deleted image: /content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/images_png/3554.png
Deleted image: /content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/images_png/3552.png
Deleted image: /content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/images_png/3772.png
Deleted image: /content/drive/MyDrive/SFA-Net-main/data/LoveDA/Val/Urban/images_png/3539.png
Deleted image: /content/drive/

In [ ]:
from PIL import Image, UnidentifiedImageError
import os
from tqdm import tqdm

def remove_corrupt_images(root_dir):
    deleted = 0
    for subdir, _, files in os.walk(root_dir):
        for file in tqdm(files):
            if file.endswith(".png"):
                path = os.path.join(subdir, file)
                try:
                    img = Image.open(path)
                    img.verify()  # Validate the image file
                except (UnidentifiedImageError, IOError):
                    print(f"Deleting corrupted image: {path}")
                    os.remove(path)
                    deleted += 1
    print(f"Done. Removed {deleted} broken files.")

# Run it on the masks
# remove_corrupt_images("data/LoveDA/Val/Urban/masks_png_convert")


  1%|          | 8/663 [00:05<07:11,  1.52it/s]


KeyboardInterrupt: 

In [ ]:
remove_corrupt_images("data/LoveDA")


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  1%|          | 16/1366 [00:15<21:52,  1.03it/s]  


KeyboardInterrupt: 